In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical

# Load and preprocess the dataset
def load_and_preprocess_data():
    # Load data
    X_train = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/X_train.txt")
    y_train = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/y_train.txt")
    X_test = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/X_test.txt")
    y_test = np.loadtxt("../../Datasets/CAPP Dataset/SubjectDependent50PercentOverlap/y_test.txt")

    # Reshape data (assuming 100 time steps and 9 features)
    X_train = X_train.reshape(X_train.shape[0], 100, 9)
    X_test = X_test.reshape(X_test.shape[0], 100, 9)

    # Normalize features
    scaler = StandardScaler()
    X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
    X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])
    X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)
    X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test.shape)

    # Manually flatten the input data
    X_train_flattened = X_train_scaled.reshape(X_train_scaled.shape[0], -1)  # (samples, 900)
    X_test_flattened = X_test_scaled.reshape(X_test_scaled.shape[0], -1)     # (samples, 900)

    # Find the actual number of unique classes
    all_labels = np.unique(np.concatenate((y_train, y_test)))
    num_classes = len(all_labels)
    print(f"Unique labels: {all_labels}")
    print(f"Number of classes: {num_classes}")

    # Create a mapping from original labels to consecutive integers
    label_map = {label: i for i, label in enumerate(all_labels)}

    # Apply the mapping to y_train and y_test
    y_train_mapped = np.array([label_map[label] for label in y_train])
    y_test_mapped = np.array([label_map[label] for label in y_test])

    # One-hot encode the mapped labels
    y_train_onehot = to_categorical(y_train_mapped, num_classes=num_classes)
    y_test_onehot = to_categorical(y_test_mapped, num_classes=num_classes)

    print(f"One-hot encoded label shape: {y_train_onehot.shape[1]}")

    return X_train_flattened, y_train_onehot, X_test_flattened, y_test_onehot, num_classes, label_map

# Create a simple MLP model for TFLite Micro
def create_mlp_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=input_shape),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Train the model
def train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=32):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train, y_train,
                        validation_data=(X_val, y_val),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping],
                        verbose=1)
    return history

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test accuracy: {accuracy:.4f}")

    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)

    # Convert back to original labels for the classification report
    inv_label_map = {v: k for k, v in label_map.items()}
    y_pred_original = np.array([inv_label_map[label] for label in y_pred_classes])
    y_test_original = np.array([inv_label_map[label] for label in y_test_classes])

    print(classification_report(y_test_original, y_pred_original, digits=5))

# Convert to TFLite
def convert_to_tflite(model, X_test):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8
    converter.inference_output_type = tf.int8

    def representative_dataset_gen():
        for input_value in tf.data.Dataset.from_tensor_slices(X_test).batch(1).take(100):
            yield [input_value]

    converter.representative_dataset = representative_dataset_gen
    tflite_model = converter.convert()

    with open('subject_dependent_mlp_tflite_model.tflite', 'wb') as f:
        f.write(tflite_model)
    print("TFLite model saved as 'subject_dependent_mlp_tflite_model.tflite'")

# Load and preprocess data
X_train, y_train, X_test, y_test, num_classes, label_map = load_and_preprocess_data()
print("num_classes: ", num_classes)

# Split training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Create and train the MLP model
input_shape = (X_train.shape[1],)  # Input shape is now 1D (900,)
MLP = create_mlp_model(input_shape, num_classes)
history = train_model(MLP, X_train, y_train, X_val, y_val)

# Evaluate the model
evaluate_model(MLP, X_test, y_test)

2024-09-27 16:24:21.002418: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 16:24:21.072705: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 16:24:21.089707: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 16:24:21.187416: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-27 16:24:21.894509: W tensorflow/compiler/tf2

Unique labels: [ 0.  1.  2.  3.  6.  7.  8.  9. 10. 11. 12. 13. 14. 19. 20. 21. 22. 23.
 25. 26. 27.]
Number of classes: 21
One-hot encoded label shape: 21
num_classes:  21


/home/denuvo-drm/miniconda3/envs/CompositeADLRecognition/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1727432686.162914   10688 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727432686.272482   10688 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727432686.

Epoch 1/50


I0000 00:00:1727432687.496586   11054 service.cc:146] XLA service 0x7478c0005f40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727432687.496611   11054 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
2024-09-27 16:24:47.526519: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-27 16:24:47.642087: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-09-27 16:24:48.016606: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_391', 8 bytes spill stores, 8 bytes spill loads



 150/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.2418 - loss: 2.5081

I0000 00:00:1727432688.692415   11054 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2390/2390 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.5435 - loss: 1.4326 - val_accuracy: 0.7320 - val_loss: 0.8092
Epoch 2/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 824us/step - accuracy: 0.7528 - loss: 0.7338 - val_accuracy: 0.7647 - val_loss: 0.7115
Epoch 3/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 783us/step - accuracy: 0.7967 - loss: 0.5949 - val_accuracy: 0.8012 - val_loss: 0.6225
Epoch 4/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 887us/step - accuracy: 0.8172 - loss: 0.5341 - val_accuracy: 0.8098 - val_loss: 0.5949
Epoch 5/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 813us/step - accuracy: 0.8389 - loss: 0.4771 - val_accuracy: 0.8214 - val_loss: 0.5619
Epoch 6/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 843us/step - accuracy: 0.8493 - loss: 0.4376 - val_accuracy: 0.8298 - val_loss: 0.5536
Epoch 7/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 782us/step - accuracy: 0.8574 - loss: 0.4164 - val_accuracy: 0.8320 - val_loss: 0.5547
Epoch 8/50
2390/2390 ━━━━━━━━━━━━━━━━━━━━ 2s 981us/step - accuracy: 0.8676 - loss: 0.3910

In [2]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(MLP)
tflite_model = converter.convert()

# Save the TFLite model
with open("GeneratedTFLiteFilesAndOGModels/SimpleMLP.tflite", "wb") as f:
    f.write(tflite_model)

print("TensorFlow Lite model has been saved as 'SimpleMLP.tflite'")

# Optional: Save the Keras model
MLP.save("GeneratedTFLiteFilesAndOGModels/SimpleMLP.h5")
print("Keras model has been saved as 'SimpleMLP.h5'")

try:
    from everywhereml.code_generators.tensorflow import convert_model

    c_header = convert_model(MLP, X_test, y_test, model_name="SimpleMLP")

    with open("GeneratedHeaderFiles/SimpleMLP.h", "w") as file:
        file.write(c_header)

    print("C header file has been saved as GeneratedHeaderFiles/SimpleMLP.h'")
except ImportError:
    print("everywhereml library not found. Skipping C header file generation.")

INFO:tensorflow:Assets written to: /tmp/tmpqjjwqwsv/assets


INFO:tensorflow:Assets written to: /tmp/tmpqjjwqwsv/assets


Saved artifact at '/tmp/tmpqjjwqwsv'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 900), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  128069934507184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069934511232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930107296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930110816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930110464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930112400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930108352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930113984: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1727432728.432713   10688 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727432728.432726   10688 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-27 16:25:28.432919: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpqjjwqwsv
2024-09-27 16:25:28.433176: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-27 16:25:28.433184: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpqjjwqwsv
2024-09-27 16:25:28.435559: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-09-27 16:25:28.436018: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-27 16:25:28.451251: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpqjjwqwsv
2024-09-27 16:25:28.455826: I tensorflow/cc/saved_model/loader.cc

TensorFlow Lite model has been saved as 'SimpleMLP.tflite'
Keras model has been saved as 'SimpleMLP.h5'
INFO:tensorflow:Assets written to: /tmp/tmplg6r2p13/assets


INFO:tensorflow:Assets written to: /tmp/tmplg6r2p13/assets


Saved artifact at '/tmp/tmplg6r2p13'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 900), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 21), dtype=tf.float32, name=None)
Captures:
  128069934507184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069934511232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930107296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930110816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930110464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930112400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930108352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128069930113984: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1727432728.689921   10688 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1727432728.689931   10688 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-09-27 16:25:28.690036: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmplg6r2p13
2024-09-27 16:25:28.690307: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-09-27 16:25:28.690317: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmplg6r2p13
2024-09-27 16:25:28.692660: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-09-27 16:25:28.708366: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmplg6r2p13
2024-09-27 16:25:28.713146: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 23112 microseconds.


C header file has been saved as GeneratedHeaderFiles/SimpleMLP.h'
